In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
%matplotlib inline

In [41]:
comments = pd.read_json('/tmp/airflow/comments.json')
votes = pd.read_json('/tmp/airflow/votes.json')
votes_and_mautic = pd.read_json('/tmp/airflow/votes_and_mautic.json').groupby('gid').count().reset_index(level=0)
votes_and_mautic_and_analytics = pd.read_json('/tmp/airflow/votes_analytics_mautic.json')

In [74]:
votes_and_mautic_and_analytics.fillna("empty", inplace=True)
votes_and_mautic_and_analytics.groupby(['email', 'analytics_campaign', 'analytics_source', 'analytics_medium']).count().reset_index(
            level=0).reset_index(
            level=0).reset_index(
            level=0).reset_index(
            level=0).sort_values(by='criado', ascending=False)
votes_and_mautic_and_analytics.head()

,email,autor,autor_id,comentário,comentário_id,escolha,criado,mtc_email,mtc_first_name,mtc_last_name,gid,analytics_source,analytics_medium,analytics_pageview,analytics_campaign
0,994000-mautic@mail.com,994000-mautic__mail.com,1075,Caixa 2 tem que ser crime sim! Se não punir os...,666,skip,2020-04-06T15:39:12.144Z,empty,empty,empty,,empty,empty,empty,empty
1,994000-mautic@mail.com,994000-mautic__mail.com,1075,Aumentar a pena para casos de corrupção e exig...,673,agree,2020-04-06T15:39:22.575Z,empty,empty,empty,,empty,empty,empty,empty
2,994000-mautic@mail.com,994000-mautic__mail.com,1075,Cadeia por muito tempo e obrigação de devolver...,672,skip,2020-04-06T15:39:28.687Z,empty,empty,empty,,empty,empty,empty,empty
3,989978-mautic@mail.com,989978-mautic__mail.com,1079,Tornar o Caixa 2 crime ajudará a evitar essa p...,667,skip,2020-04-06T15:42:46.771Z,empty,empty,empty,,empty,empty,empty,empty
4,964533-mautic@mail.com,964533-mautic__mail.com,1045,É preciso muito cuidado com a Lei de Abuso de ...,670,agree,2020-04-06T23:27:07.392Z,empty,empty,empty,GA1.3.1145770264.1593625413,empty,empty,empty,empty
